In [1]:
import pandas as pd
import numpy as np

In [2]:
class CarboneAlpha:
    """Classe pour représenter un carbone alpha d'une protéine."""
    
    def __init__(self, number, x, y, z):
        self.number = number
        self.x = x
        self.y = y
        self.z = z
    
    def compute_distance(self, other):
        dist = ((other.x - self.x) ** 2 + (other.y - self.y) ** 2 + (other.z - self.z) ** 2) ** 0.5
        return dist

class Template:
    """Classe pour représenter le template utilisé."""
    
    def __init__(self, file):
        self.structure = self.build_template_from_pdb(file)
        self.length = len(self.structure)

    def build_template_from_pdb(self, filename):
        list_calpha = []
        with open(filename, "r") as pdb :
            for ligne in pdb:
                if ligne.startswith("ATOM") and (ligne[12:16].strip() == "CA"):
                    number = ligne[6:11].strip()
                    x = float(ligne[30:38].strip())
                    y = float(ligne[38:46].strip())
                    z = float(ligne[46:54].strip())
                                       
                    list_calpha.append(CarboneAlpha(number, x, y, z))
        return list_calpha
        
    def build_dist_matrix(self):
        dist_list = []
        
        for i, atom in enumerate(self.structure):
            dist_ligne = []
            for other in (self.structure):
                dist_ligne.append(atom.compute_distance(other))
            dist_list.append(dist_ligne)
            
        dist_matrix = np.array(dist_list)
        return dist_matrix
    
    def __str__(self):
        string = ""
        for i, ca in enumerate(self.structure):
            string += f"position {i}-{ca.number}, coor( {ca.x}, {ca.y}, {ca.z})\n"
        return string

In [3]:
PDB_FILE = "../data/5awl.pdb"
TEMPLATE = Template(PDB_FILE)
print(TEMPLATE)
DIST_MATRIX = TEMPLATE.build_dist_matrix()
print(DIST_MATRIX)

position 0-2, coor( 24.935, 20.652, 10.774)
position 1-14, coor( 21.801, 19.381, 9.017)
position 2-26, coor( 18.477, 20.539, 10.432)
position 3-34, coor( 16.223, 17.66, 11.537)
position 4-41, coor( 13.04, 19.289, 10.212)
position 5-50, coor( 14.2, 20.806, 6.941)
position 6-57, coor( 17.256, 18.676, 6.039)
position 7-61, coor( 19.242, 21.76, 5.125)
position 8-68, coor( 22.762, 22.68, 6.241)
position 9-82, coor( 23.135, 25.467, 8.794)

[[ 0.          3.81109512  6.46803656  9.24300692 11.98601844 11.3998171
   9.23535825  8.09624073  5.42059056  5.50859556]
 [ 3.81109512  0.          3.7937007   6.35816994  8.84260199  8.00722187
   5.47928225  5.2302759   4.41736324  6.2344752 ]
 [ 6.46803656  3.7937007   0.          3.81970967  5.58317732  5.52730486
   4.92545013  5.49911947  6.36471421  6.97604415]
 [ 9.24300692  6.35816994  3.81970967  0.          3.81323419  5.92562748
   5.68571447  8.18768007  9.79829256 10.7818849 ]
 [11.98601844  8.84260199  5.58317732  3.81323419  0.          

In [4]:
def clean_DOPE_data(filename):
    ca_matrix = []
    
    with open(filename, "r") as dope :
        for ligne in dope:
            if ligne[3:7].strip() == "CA" and ligne[11:14].strip() == "CA":
                ca_matrix.append(ligne.split())
    
    columns = ['res1', 'temp1', 'res2', 'temp2'] + list(np.arange(0.25, 15, 0.5))
    dope_score = pd.DataFrame(ca_matrix, columns = columns) 
    dope_score = dope_score.drop(['temp1', 'temp2'], axis=1)
    
    return dope_score

In [5]:
DOPE_FILE = "../data/dope.par"
DOPE_MATRIX = clean_DOPE_data(DOPE_FILE)
print(DOPE_MATRIX)

    res1 res2   0.25   0.75   1.25   1.75   2.25   2.75   3.25   3.75  ...  \
0    ALA  ALA  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.64  ...   
1    ALA  ARG  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.61  ...   
2    ALA  ASN  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.58  ...   
3    ALA  ASP  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.62  ...   
4    ALA  CYS  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.49  ...   
..   ...  ...    ...    ...    ...    ...    ...    ...    ...    ...  ...   
395  VAL  SER  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.48  ...   
396  VAL  THR  10.00  10.00  10.00  10.00  10.00   5.84   6.15  -1.56  ...   
397  VAL  TRP  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.27  ...   
398  VAL  TYR  10.00  10.00  10.00  10.00  10.00  10.00   5.71  -1.38  ...   
399  VAL  VAL  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.48  ...   

     10.25  10.75  11.25  11.75  12.25  12.75  13.25  13.75  14

# Classes matrices dynamiques

In [54]:
class DynamicMatrix:    
    def __init__(self, lines, columns, gap):
        self.matrix = np.zeros((lines, columns))
        self.lines = lines
        self.columns = columns
        self.gap = gap

    def initialize_matrix(self, first_val, start, end, get_score):
        if (start[0] < 0) or (start[1] < 0):
            raise ValueError("Start of initialization out of matrix.")
        if (end[0] >= self.lines) or (end[1] >= self.columns):
            raise ValueError("End of initialization out of matrix.")
        
        # Première case
        self.matrix[start[0], start[1]] = first_val
        
        # Remplissage de la première colonne jusqu'à la limite
        for i in range(start[0] + 1, end[0] + 1):
            self.matrix[i, start[1]] = self.matrix[i - 1, start[1]] + self.gap + get_score(i, start[1])

        # Remplissage de la première ligne jusqu'à la limite
        for j in range(start[1] + 1, end[1] + 1):
            self.matrix[start[0], j] = self.matrix[start[0], j - 1] + self.gap + get_score(start[0], j)

In [100]:
class LowLevelMatrix(DynamicMatrix):
    aa_codes = {
    'A': 'ALA', 'R': 'ARG', 'N': 'ASN', 'D': 'ASP', 'C': 'CYS',
    'Q': 'GLN', 'E': 'GLU', 'G': 'GLY', 'H': 'HIS', 'I': 'ILE',
    'L': 'LEU', 'K': 'LYS', 'M': 'MET', 'F': 'PHE', 'P': 'PRO',
    'S': 'SER', 'T': 'THR', 'W': 'TRP', 'Y': 'TYR', 'V': 'VAL'
    }
    
    def __init__(self, gap, frozen, distance, dope, sequence):
        lines = len(sequence)
        columns = len(distance)
        
        DynamicMatrix.__init__(self, lines, columns, gap)

        # Vérification du blocage de la case
        if (frozen['seq_id'] >= lines) or (frozen['seq_id'] < 0):
            raise ValueError("Frozen line index out of matrix.")
        if (frozen['pos_id'] >= columns) or (frozen['pos_id'] < 0):
            raise ValueError("Frozen column index out of matrix")

        # Récupération du résidu fixé
        frozen['seq_res'] = sequence[frozen['seq_id']]
        
        self.frozen = frozen
        self.distance = distance
        self.dope = dope
        self.sequence = sequence

    def round_distance(self, dist):
        # arrondi au quart le plus proche
        rounded_value = round(dist * 4) / 4
        
        # ne garde que 0.25 ou 0.75
        decimal = rounded_value % 1
        if decimal == 0.0:
            return rounded_value + 0.25
        elif decimal == 0.5:
            return rounded_value + 0.25
        else:
            return rounded_value
    
    def get_score(self, i, j):
        dist = self.distance[self.frozen["pos_id"], j]
        closest_dist = self.round_distance(dist)
        score = self.dope.loc[(self.dope['res1'] == self.aa_codes[self.frozen['seq_res']]) & (self.dope['res2'] == self.aa_codes[self.sequence[i]]), closest_dist]
        
        return float(score.values[0])
    
    def fill_matrix(self):
        # Partie supérieure gauche
        self.initialize_matrix(0, [0, 0], [self.frozen['seq_id'] - 1, self.frozen['pos_id'] - 1], self.get_score)

        for i in range(1, self.frozen['seq_id']):
            for j in range(1, self.frozen['pos_id']):
                score = self.get_score(i, j)
                self.matrix[i, j] = score + min(self.matrix[i - 1, j - 1],
                                                self.matrix[i - 1, j] + self.gap,
                                                self.matrix[i, j - 1] + self.gap
                                               )

        # Case fixée
        if (self.frozen['seq_id'] == 0):
            self.matrix[self.frozen['seq_id'], self.frozen['pos_id']] = self.matrix[self.frozen['seq_id'], self.frozen['pos_id'] - 1]
        elif (self.frozen['pos_id'] == 0):
            self.matrix[self.frozen['seq_id'], self.frozen['pos_id']] = self.matrix[self.frozen['seq_id'] - 1, self.frozen['pos_id']]
        else :
            self.matrix[self.frozen['seq_id'], self.frozen['pos_id']] = self.matrix[self.frozen['seq_id'] - 1, self.frozen['pos_id'] - 1]

        # Partie inférieure droite (si elle existe)
        if (self.frozen['seq_id'] != self.lines - 1) and (self.frozen['pos_id'] != self.columns - 1):
            self.initialize_matrix(self.matrix[self.frozen['seq_id'], self.frozen['pos_id']],
                                   [self.frozen['seq_id'] + 1, self.frozen['pos_id'] + 1],
                                   [self.lines - 1, self.columns - 1], self.get_score
                                  )
    
            for i in range(self.frozen['seq_id'] + 2, self.lines):
                for j in range(self.frozen['pos_id'] + 2, self.columns):
                    score = self.get_score(i, j)
                    self.matrix[i, j] = score + min(self.matrix[i - 1, j - 1],
                                                    self.matrix[i - 1, j] + self.gap,
                                                    self.matrix[i, j - 1] + self.gap
                                                   )
            max_score = self.matrix[i, j]
        else :
            max_score = self.matrix[self.frozen['seq_id'], self.frozen['pos_id']]
        
        return max_score

In [102]:
FROZEN = {'seq_id': 5, 'pos_id': 5}
SEQUENCE = "YYDPETGTWY"
l_test = LowLevelMatrix(0, FROZEN, DIST_MATRIX, DOPE_MATRIX, SEQUENCE)

print(l_test.matrix)
MAX_SCORE = l_test.fill_matrix()
print(l_test.matrix)
print(MAX_SCORE)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[ 0.000e+00  1.700e-01 -2.500e-01 -5.300e-01 -2.070e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [-3.000e-02  1.400e-01 -6.700e-01 -9.500e-01 -3.610e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e-02  2.000e-01 -1.260e+00 -1.440e+00 -5.200e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e-01  1.300e-01 -1.670e+00 -2.070e+00 -6.930e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.500e-01  3.000e-01 -1.980e+00 -2.290e+00 -8.570e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00 -8.570e+00
   0.000e+00  0.000e+00  0

In [66]:
frozen = {'seq_id': 2, 'pos_id': 2}
aa_codes = {
    'A': 'ALA', 'R': 'ARG', 'N': 'ASN', 'D': 'ASP', 'C': 'CYS',
    'Q': 'GLN', 'E': 'GLU', 'G': 'GLY', 'H': 'HIS', 'I': 'ILE',
    'L': 'LEU', 'K': 'LYS', 'M': 'MET', 'F': 'PHE', 'P': 'PRO',
    'S': 'SER', 'T': 'THR', 'W': 'TRP', 'Y': 'TYR', 'V': 'VAL'
    }

In [61]:
frozen['seq_res'] = 'A'
print(frozen)

{'seq_id': 2, 'pos_id': 2, 'seq_res': 'A'}


In [62]:
DOPE_MATRIX.loc[(DOPE_MATRIX['res1'] == aa_codes['A']) & (DOPE_MATRIX['res2'] == aa_codes[SEQUENCE[5]]), 8.25]

16    0.25
Name: 8.25, dtype: object

In [13]:
print(TEMPLATE.length)

10


In [15]:
print(len(DIST_MATRIX))

10
